In [43]:
from collections import Counter
import json
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import torch

# Use CPU for now
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [52]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

In [53]:
# Preprocessing function
def preprocess(example):
    input_ = f"generate title: {example['description']}"
    print(input_)
    target = example["name"]
    print(target)
    

    inputs = tokenizer(input_, max_length=100, truncation=True, padding="max_length")
    targets = tokenizer(target, max_length=10, truncation=True, padding="max_length")

    inputs["labels"] = targets["input_ids"]
    return inputs

# Load dataset
with open("climbing_routes.json", "r") as json_file:
    data = json.load(json_file)
print("Example:", data[0])

dataset = Dataset.from_list(data)
print("Dataset size:", len(dataset))

tokenized_dataset = dataset.map(preprocess, remove_columns=["name", "grade", "description"])
print("Tokenized dataset size:", len(tokenized_dataset))
print("Example:", tokenized_dataset[0])




Example: {'name': 'Jurassic Climb', 'grade': 'V1', 'description': 'A pleasant introduction to the crag, featuring easy holds and a straightforward ascent.'}
Dataset size: 100


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

generate title: A pleasant introduction to the crag, featuring easy holds and a straightforward ascent.
Jurassic Climb
generate title: Short but dynamic, with a crux move involving a high step and a reach for a pocket.
Chalk Dust
generate title: A series of fluid moves on solid holds, with a technical crux near the top.
Boulder Dash
generate title: A powerful start with a technical traverse, leading to a juggy finish.
Grit & Glory
generate title: A delicate balance of power and finesse, with a tricky heel hook required to reach the top.
Crimp & Punish
generate title: A crack climb with technical footwork and a dynamic finish.
The Crack of Dawn
generate title: A power-endurance test with sustained crimping and a technical crux near the top.
Rock Bottom
generate title: A technical start with a powerful dyno midway, requiring precise footwork.
Chalk Monster
generate title: A steep, overhanging route with powerful moves and a technical crux near the top.
Boulder of the Gods
generate title:

In [54]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    use_mps_device=True,
    per_device_train_batch_size=4,
    num_train_epochs=20,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    evaluation_strategy="no",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()


/opt/anaconda3/envs/climber/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/anaconda3/envs/climber/lib/python3.8/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


  0%|          | 0/500 [00:00<?, ?it/s]

{'loss': 8.8292, 'grad_norm': 50.54391098022461, 'learning_rate': 4.9e-05, 'epoch': 0.4}
{'loss': 5.945, 'grad_norm': 30.938169479370117, 'learning_rate': 4.8e-05, 'epoch': 0.8}
{'loss': 4.8184, 'grad_norm': 16.586227416992188, 'learning_rate': 4.7e-05, 'epoch': 1.2}
{'loss': 4.3443, 'grad_norm': 17.495145797729492, 'learning_rate': 4.600000000000001e-05, 'epoch': 1.6}
{'loss': 4.0815, 'grad_norm': 8.681951522827148, 'learning_rate': 4.5e-05, 'epoch': 2.0}
{'loss': 3.5511, 'grad_norm': 8.355608940124512, 'learning_rate': 4.4000000000000006e-05, 'epoch': 2.4}
{'loss': 3.1048, 'grad_norm': 12.069748878479004, 'learning_rate': 4.3e-05, 'epoch': 2.8}
{'loss': 3.055, 'grad_norm': 9.588069915771484, 'learning_rate': 4.2e-05, 'epoch': 3.2}
{'loss': 2.5131, 'grad_norm': 6.702809810638428, 'learning_rate': 4.1e-05, 'epoch': 3.6}
{'loss': 2.5643, 'grad_norm': 12.065796852111816, 'learning_rate': 4e-05, 'epoch': 4.0}
{'loss': 2.1028, 'grad_norm': 5.87778902053833, 'learning_rate': 3.9000000000000

TrainOutput(global_step=500, training_loss=1.6619061813354492, metrics={'train_runtime': 82.9639, 'train_samples_per_second': 24.107, 'train_steps_per_second': 6.027, 'total_flos': 52867891200000.0, 'train_loss': 1.6619061813354492, 'epoch': 20.0})

In [55]:

# Generate function
def generate_title(description):
    input_text = f"generate title: {description}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=100, truncation=True).to(device)
    output_ids = model.generate(input_ids, max_length=10, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Test generation
print(generate_title(
    "A steep and powerful boulder with a crimp in the middle and a crux move to a good hold."
))

print(generate_title(
    "A hard start on a crimp with a heel hook, then a big move with the right hand to a good hold."
))

Rock & Roll
Rock & Roll
